In [1]:
import psycopg2 as pg2
import pandas as pd
import glob
from tqdm import tqdm
tqdm.pandas()
import gc

### Create tables

In [2]:
class Database():
    def __init__(self,df_tmp, name_table):

        self.df_ = df_tmp
        self.name_table = name_table
        print(self.name_table)
        columns=[col for col in self.df_.columns if col not in ['references']]
        self.df_ = self.df_[columns]
        columns = {col:'TEXT' for col in self.df_.columns}
        columns.update({'id':'SERIAL PRIMARY KEY'})
        self.columns = columns
        self.con = pg2.connect(host='aminer_postgres_db',
                  user='postgres',
                  password='password',
                  database='postgres')
        self.con.autocommit = True
        self.cur = self.con.cursor()
    
    # запросы в бд
    def request_db(self, sql):
        return pd.read_sql(sql,self.con)
    
    # создание таблицы
    def create_paper(self):
        sub_q = ', '.join(' '.join((k, self.columns[k])) for k,v in self.columns.items())
        q = f'''CREATE TABLE IF NOT EXISTS {self.name_table} ({sub_q});'''
        print(q)
        self.cur.execute(q)
        self.delete_all_rows()
    
    # удаление таблицы
    def empty(self):
        q=f"DROP table IF EXISTS {self.name_table};"
        self.request_db(q)

    # удаление 1ой строчки
    def delete_row(self,id_):
        q=f"DELETE FROM {self.name_table} WHERE id=='{id_}'"
        self.request_db(q)

    # удаление нескольких строк
    def delete_many_rows(self,id_list):
        id_tuple = tuple(id_ for id_ in id_list)
        q=f"DELETE FROM {self.name_table} WHERE id in {id_tuple}"
        self.request_db(q)
        
    # удаление нескольких строк без удаления самой таблицы
    def delete_all_rows(self):
        q=f"DELETE FROM {self.name_table}"
        self.cur.execute(q)
    
    # сохрание в бд
    def record_data(self, dfsm):
        for index, dic in tqdm(dfsm.iterrows()):
            values = []
            for col in dfsm.columns:
                if self.columns[col] == 'TEXT':
                    values.append(str(dic[col]))
                else: 
                    values.append(int(float(dic[col])))
            data_tuple = list(values)
            ques = data_tuple
            for i, qu in enumerate(ques):
                if qu == None or qu == 'nan':
                    ques[i] = ''
                else:
                    ques[i] = ques[i].replace('\'', '')
            sub_q = ', '.join(col for col in list(dfsm.columns))
            q=f'''INSERT INTO {self.name_table}({sub_q}) VALUES {tuple(ques)};'''
            self.cur.execute(q)
        
#             if index % 100 == 0:
#                 print(f'{len(dfsm) - index} rows left')
#                 pass
        return True

    def insertion(self,dfsm):
        try:
            return self.record_data(dfsm)
        except Exception as error:
            print(f'{error=}')
            return False

    def insert_all(self):
        err_list = []
        
        dfsm = self.df_
        
        rt = self.insertion(dfsm)
        return err_list
   
    # метод для проверки бд
    def select_head(self):
        q = f'select * from {self.name_table} limit 10'
        return pd.read_sql(q, self.con)
    

### Read files

In [3]:
for file in glob.glob('../data/data_csv/*'):
    df_tmp = pd.read_csv(file)
    # ограничил 1000 строчек
    df_tmp = df_tmp[:1000]
    ###################################
    idx_file = 'Paper_' + file.split('/')[-1].split('.')[0].split('_')[-1]
    db = Database(df_tmp, idx_file)
    db.create_paper()
    err_list = db.insert_all()
    del df_tmp
    gc.collect()


Paper_00


0it [00:00, ?it/s]

CREATE TABLE IF NOT EXISTS Paper_00 (_id TEXT, title TEXT, venue TEXT, year TEXT, keywords TEXT, n_citation TEXT, lang TEXT, authors TEXT, fos TEXT, page_start TEXT, page_end TEXT, volume TEXT, issue TEXT, issn TEXT, isbn TEXT, doi TEXT, pdf TEXT, url TEXT, abstract TEXT, id SERIAL PRIMARY KEY);


1000it [00:02, 372.99it/s]


In [4]:
### Check db
db.select_head()

,_id,title,venue,year,keywords,n_citation,lang,authors,fos,page_start,page_end,volume,issue,issn,isbn,doi,pdf,url,abstract,id
0,53e99784b7602d9701f3f996,FEAL.,{raw: Encyclopedia of Cryptography and Securit...,2011.0,[],0.0,en,"[{_id: 54055d53dabfae91d3fd9f5a, name: Christo...",,454,455,,,,,,,[http://dx.doi.org/10.1007/978-1-4419-5906-5_576],,1084
1,53e99784b7602d9701f3f72b,Forschungsmethoden,"{_id: 53a72f4820f7420be8c9d9b2, raw: CSCL-Komp...",2004.0,[],1.0,en,"[{sid: 235799, _id: 53f32548dabfae9a8446d9b8, ...",,5,13,,,,,,,,,1178
2,53e99784b7602d9701f3f72c,Fehlertoleranzverfahren,,1990.0,[],122.0,en,"[{_id: 53f3887cdabfae4b34a21d76, name: Klaus E...",,,,,,,isbn,,,,,1179
3,53e99784b7602d9701f3f65a,FaceCerts,"{_id: 53a72a3f20f7420be8bf81bf, raw: DCC}",2003.0,[],4.0,en,"[{_id: 562d00d345cedb3398d26ba3, name: Darko K...",,435,435,,,,0-7695-1896-6,,,[http://doi.ieeecomputersociety.org/10.1109/DC...,,1116
4,53e99785b7602d9701f42c68,Einwurf,"{_id: 53a7315020f7420be8d385f9, raw: HMD - Pra...",2003.0,[],0.0,en,"[{_id: 53f43fefdabfaedf435baab1, name: Hans-Pe...",,,,230,3,,,,,"[http://hmd.dpunkt.de/231/einwurf.html, http:/...",,1752
5,53e99785b7602d9701f42c69,Einwurf,"{_id: 53a7315020f7420be8d385f9, raw: HMD - Pra...",2003.0,[],0.0,en,"[{name: Gunter Dueck, sid: 2621379, _id: 54314...",,,,231,,,,,,[http://hmd.dpunkt.de/232/einwurf.html],,1753
6,53e99784b7602d9701f3f65e,Fairytales,"{_id: 5736ad96d39c4f40a796c5f9, raw: SIGART Ne...",1976.0,[],16.0,en,"[{name: Allen Newell, org: Carnegie-Mellon Uni...",,6,7,60,60,,,10.1145/1045276.1045277,,"[http://dx.doi.org/10.1145/1045276.1045277, ht...",,1117
7,53e99785b7602d9701f42c6a,Einwurf,"{_id: 53a7315020f7420be8d385f9, raw: HMD - Pra...",2001.0,[],0.0,en,"[{_id: 53f43a1ddabfaeee229cd684, name: Helmut ...",,,,219,,,,,,[http://hmd.dpunkt.de/219/einwurf.html],,1754
8,53e99785b7602d9701f4084f,GammaPolarSlicer.,"{_id: 555036f67cea80f95416aa0d, raw: Comput. S...",2011.0,[],0.0,en,"[{_id: 53f37e0cdabfae4b349ecc87, name: Sergio ...",,477,499,8,2,,,,,[http://dx.doi.org/10.2298/CSIS110107006A],,1228
9,53e99785b7602d9701f40850,GamesScience,{type: 0},2006.0,[],0.0,en,,,92,92,18,2,,,10.1026/1617-6383.18.2.92,,,,1229
